# Question Answering with Wikipedia Pages

Welcome to your question-answering adventure! In this notebook, we'll build a system that can fetch entire Wikipedia pages and answer questions about them using pre-trained language models.

## What You'll Learn
- How to fetch and process Wikipedia pages
- How to use pre-trained BERT models for question answering on long documents
- How to build a practical Q&A system for real-world content
- How to handle large text documents that exceed model context limits

## Setup and Imports

First, let's install and import the necessary libraries. We'll use Wikipedia API to fetch pages and Transformers for Q&A.

In [ ]:
# Install required packages
%pip install transformers torch wikipedia

In [ ]:
# Import necessary libraries
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import wikipedia
import torch
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## Load the Question Answering Model

Let's load a pre-trained model optimized for question answering. We'll use DistilBERT for efficiency.

In [ ]:
# Load the question-answering pipeline
qa_pipeline = pipeline(
    "question-answering",
    model="distilbert-base-cased-distilled-squad",
    tokenizer="distilbert-base-cased-distilled-squad"
)

print("Question-answering model loaded successfully!")
print("Model: DistilBERT fine-tuned on SQuAD dataset")

## Wikipedia Q&A System

Now let's build the core system that can fetch Wikipedia pages and answer questions about them.

In [ ]:
def answer_question(context, question):
    """
    Answer a question based on the given context.
    
    Args:
        context (str): The text passage containing the information
        question (str): The question to answer
    
    Returns:
        dict: Contains the answer, confidence score, and position
    """
    result = qa_pipeline(question=question, context=context)
    return result

def fetch_wikipedia_page(title):
    """Fetch Wikipedia page content by title."""
    try:
        wikipedia.set_lang("en")
        page = wikipedia.page(title)
        return page.content
    except Exception as e:
        return None

def split_text_into_chunks(text, max_length=400):
    """Split text into chunks that fit within model limits."""
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_length * 4:
            current_chunk += sentence + ". "
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

def answer_question_from_wikipedia(page_title, question):
    """Answer a question using content from a Wikipedia page."""
    # Fetch the page
    content = fetch_wikipedia_page(page_title)
    if not content:
        return {"error": "Could not fetch Wikipedia page"}
    
    # Split into chunks
    chunks = split_text_into_chunks(content)
    
    # Find the best answer across all chunks
    best_answer = None
    best_score = 0.0  # Explicitly initialize as float
    best_chunk_idx = -1
    best_result = None
    
    for i, chunk in enumerate(chunks):
        try:
            result = qa_pipeline(question=question, context=chunk)
            # Extract score and ensure it's a proper float
            score = float(result['score'])
            
            # Validate score is in expected range [0, 1]
            if score < 0 or score > 1:
                print(f"WARNING: Score {score} is outside [0,1] range in chunk {i}")
                continue
            
            if score > best_score:
                best_score = score
                best_answer = result['answer']
                best_chunk_idx = i
                best_result = result
        except Exception as e:
            print(f"Error processing chunk {i}: {e}")
            continue
    
    # Final validation of best score
    if best_score > 1.0:
        print(f"ERROR: Final best_score {best_score} is > 1.0! This shouldn't happen.")
        best_score = min(best_score, 1.0)  # Clamp to 1.0 as safety measure
    
    return {
        'question': question,
        'answer': best_answer,
        'confidence': best_score,
        'chunk_index': best_chunk_idx,
        'total_chunks': len(chunks)
    }

print("Q&A system ready!")
print("You can now use:")
print("- answer_question(context, question) for basic Q&A")
print("- answer_question_from_wikipedia(page_title, question) for Wikipedia Q&A")

## Test the Q&A System

Let's test the basic Q&A function with different types of questions.

In [ ]:
# Test basic Q&A with a simple example
test_context = """
The Transformer architecture was introduced in the paper "Attention Is All You Need" by Vaswani et al. in 2017. 
It revolutionized natural language processing by using self-attention mechanisms instead of recurrent neural networks. 
The model consists of an encoder and decoder, each made up of multiple layers with multi-head attention and feed-forward networks.
"""

test_question = "When was the Transformer architecture introduced?"

print("=== BASIC Q&A TEST ===")
result = answer_question(test_context, test_question)
print(f"Question: {test_question}")
print(f"Answer: {result['answer']}")
print(f"Confidence: {result['score']:.4f}")
print(f"Position: {result['start']}-{result['end']}")
print("-" * 50)

In [ ]:
# Test different types of questions on a more complex context
llm_context = """
Large Language Models (LLMs) are AI systems trained on vast amounts of text data to understand and generate human-like text. 
Popular examples include GPT-3, GPT-4, BERT, and T5. These models use transformer architectures and are pre-trained on 
diverse internet text before being fine-tuned for specific tasks. LLMs can perform various tasks including text generation, 
question answering, summarization, and translation. They typically have billions of parameters and require significant 
computational resources for training and inference. The training process involves predicting the next word in a sequence, 
which helps the model learn language patterns, grammar, and world knowledge.
"""

questions = [
    "What are some examples of Large Language Models?",
    "What architecture do LLMs use?",
    "How many parameters do LLMs typically have?",
    "What tasks can LLMs perform?",
    "What is the training process for LLMs?"
]

print("=== TESTING DIFFERENT QUESTION TYPES ===")
for i, question in enumerate(questions, 1):
    print(f"\nQuestion {i}: {question}")
    result = answer_question(llm_context, question)
    print(f"Answer: {result['answer']}")
    print(f"Confidence: {result['score']:.4f}")
    print(f"Position: {result['start']}-{result['end']}")
    print("-" * 50)

## Interactive Q&A Session

Now let's create an interactive session where you can ask questions about any text passage.

In [ ]:
def interactive_qa():
    """
    Interactive question-answering session.
    """
    print("Interactive Q&A Session")
    print("=" * 30)
    print("Enter your context (text passage) and questions.")
    print("Type 'quit' to exit, 'new context' to change the passage.")
    
    # Default context about AI and Machine Learning
    context = """
    Artificial Intelligence (AI) is a branch of computer science that aims to create machines capable of intelligent behavior. 
    Machine Learning (ML) is a subset of AI that enables computers to learn and improve from experience without being explicitly programmed. 
    Deep Learning is a subset of ML that uses neural networks with multiple layers to model and understand complex patterns in data. 
    Natural Language Processing (NLP) is a field of AI that focuses on the interaction between computers and human language, 
    enabling machines to understand, interpret, and generate human language.
    """
    
    print(f"\nCurrent context: {context[:100]}...")
    
    while True:
        question = input("\nEnter your question (or 'quit' to exit): ")
        
        if question.lower() == 'quit':
            print("Goodbye!")
            break
        elif question.lower() == 'new context':
            context = input("Enter new context: ")
            print(f"Context updated: {context[:100]}...")
            continue
        
        if question.strip():
            try:
                result = answer_question(context, question)
                print(f"\nQuestion: {question}")
                print(f"Answer: {result['answer']}")
                print(f"Confidence: {result['score']:.4f}")
                print(f"Position: {result['start']}-{result['end']}")
                print("-" * 50)
            except Exception as e:
                print(f"Error: {e}")
                print("Please try a different question.")

# Uncomment the line below to start interactive session
# interactive_qa()

## Your Task: Test with Space Race

**Your mission**: Use the system to answer any question using the Space Race Wikipedia page.

Run the code below and report what answer you get!

In [ ]:
# Your test - answer the specific question about the Space Race
your_question = ""
result = answer_question_from_wikipedia("Race to space", your_question)

print("YOUR RESULT:")
print("="*30)
print(f"Question: {result['question']}")
print(f"Answer: {result['answer']}")
print(f"Confidence: {result['confidence']:.4f}")
print(f"Found in chunk {result['chunk_index']} of {result['total_chunks']}")
print("\nThis is the answer you should report back!")

## Summary

Congratulations! You've successfully built a question-answering system using pre-trained language models. 

### Key Takeaways:
- **Context Understanding**: LLMs can understand context and extract precise information
- **Model Comparison**: Different models have different strengths and trade-offs
- **Real-world Applications**: Q&A systems have many practical uses
- **Experimentation**: Testing with different contexts and questions reveals model capabilities

You now have the foundation to build more sophisticated question-answering systems!